#Scrap Data

In [ ]:
#Dokumentasi https://pypi.org/project/app-store-scraper/
!pip3 install app-store-scraper

In [ ]:
# from app_store_scraper import AppStore
# import pandas as pd
# import numpy as np
# import json

# app = AppStore(country='id', app_name='myIndiHome', app_id = '1119407221')
# app.review(how_many=50000)

# df = pd.DataFrame(np.array(app.reviews),columns=['review'])
# df2 = df.join(pd.DataFrame(df.pop('review').tolist()))
# df2.head()
# df2.to_csv('Reviews.csv')

In [ ]:
from app_store_scraper import AppStore
import pandas as pd
import numpy as np
import json
from datetime import datetime

# Inisialisasi scraping untuk aplikasi myIndiHome
app = AppStore(country='id', app_name='myIndiHome', app_id='1119407221')

# Mengambil ulasan (misalnya 50.000 ulasan)
app.review(how_many=50000)

# Konversi data ulasan ke DataFrame
df = pd.DataFrame(np.array(app.reviews), columns=['review'])
df2 = df.join(pd.DataFrame(df.pop('review').tolist()))

# Filter ulasan berdasarkan rentang tanggal
start_date = datetime.strptime("2024-10-08", "%Y-%m-%d")
end_date = datetime.strptime("2024-11-29", "%Y-%m-%d")

# Pastikan kolom tanggal diubah ke format datetime untuk filter
df2['date'] = pd.to_datetime(df2['date'])

# Filter data berdasarkan rentang tanggal
filtered_df = df2[(df2['date'] >= start_date) & (df2['date'] <= end_date)]

# Simpan hasil ke CSV
filtered_df.to_csv('Reviews.csv', index=False)

print("Scraping dan filter selesai. Data disimpan di 'Reviews.csv'")

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Reviews.csv')
df.head()

In [ ]:
# Function to extract 'id', 'modified' and 'body' from 'developerResponse'
def extract_developer_response(response):
    if pd.isna(response):
        return pd.Series([None, None])
    else:
        try:
            response_dict = eval(response)
            return pd.Series([response_dict.get('id'), response_dict.get('body'),response_dict.get('modified')])
        except:
            return pd.Series([None, None])

# Apply the function and create new columns 'response_id' and 'response_body' and 'response_modified'
df[['response_id', 'response_body','response_modified']] = df['developerResponse'].apply(extract_developer_response)

# Drop the original 'developerResponse' column
df_cleaned = df.drop(columns=['developerResponse'])

# Display the cleaned dataframe
df_cleaned.head()


In [ ]:
df_cleaned['response_modified'] = pd.to_datetime(df_cleaned['response_modified'], format='%Y-%m-%dT%H:%M:%SZ', errors='coerce')

In [ ]:
df_cleaned['response_modified'] = df_cleaned['response_modified'].dt.strftime('%d-%m-%Y %H:%M:%S')

In [ ]:
df_cleaned['date'] = pd.to_datetime(df_cleaned['date'], errors='coerce')
df_cleaned['date'] = df_cleaned['date'].dt.strftime('%d-%m-%Y %H:%M:%S')

In [ ]:
df_cleaned

In [ ]:
df_cleaned['response_id'] = df_cleaned['response_id'].fillna(0).astype(int)

# Mengubah nilai 0 kembali menjadi string kosong
df_cleaned['response_id'] = df_cleaned['response_id'].replace(0, '')

In [ ]:
df_cleaned

In [ ]:
# prompt: split the date and timestamp

# Split the 'date' column into 'date' and 'time' columns
df_cleaned[['date', 'timestamp']] = df_cleaned['date'].str.split(' ', expand=True)
df_cleaned.head()


In [ ]:
# prompt: move the timestamp to after date

# Rearrange columns
df_cleaned = df_cleaned[['date', 'timestamp','userName','title', 'review','rating','isEdited', 'response_id', 'response_body', 'response_modified']]
df_cleaned.head()


# Filter date start 01-08-2024 untill 12-9-2024

In [ ]:
# prompt: filter date start 01-08-2024 untill 12-9-2024

import pandas as pd
# Convert 'date' column to datetime objects
df_cleaned['date'] = pd.to_datetime(df_cleaned['date'], format='%d-%m-%Y', errors='coerce')

# Filter the dataframe based on the date range
filtered_df = df_cleaned[
    (df_cleaned['date'] >= pd.to_datetime('08-10-2024', format='%d-%m-%Y')) &
    (df_cleaned['date'] <= pd.to_datetime('29-11-2024', format='%d-%m-%Y'))
]

# Display the filtered dataframe
filtered_df = filtered_df.reset_index(drop=True)
filtered_df


#write to sheet

In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread as gs
from gspread_dataframe import set_with_dataframe

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/MyDrive/Intern Data Telkom/Daniel/writesheet/nilproject-424807-3a1a96640896.json', scope)
gc = gs.authorize(credentials)

In [ ]:
ws = gc.open(f'User Feedback App My IndiHome').worksheet(f'App Store')
ws.clear()
set_with_dataframe(ws, filtered_df)